# process balloon dataset

- 基于提供的 notebook，将 cat 数据集换成气球数据集


In [1]:
import os
import shutil

from tqdm import tqdm

PROJECT_ROOT = "/home/louzekun/projects/mmlab/mmdet"
os.chdir(PROJECT_ROOT)
os.getcwd()

'/mnt/data/ai4phys/louzekun/projects/mmlab/mmdet'

In [2]:
!ls; ls data; ls data/balloon


balloon_RTMDet.py  data			     mmdet_test.ipynb	      tools
basic.ipynb	   logs			     mmdet_viz_dataset.ipynb
checkpoint	   mmdet_proc_dataset.ipynb  outputs
configs		   mmdet_RTMDet.ipynb	     README.md
balloon  Drink_coco
train  val


In [2]:
import json
import re
from collections import defaultdict
from tqdm import tqdm

import cv2


In [5]:
# dataset_path = "./data/balloon/train"
# jsonfname = 'balloon_train.json'
dataset_path = "./data/balloon/val"
jsonfname = 'balloon_val.json'

with open(os.path.join(dataset_path, "via_region_data.json")) as f:
    annotations = json.load(f)

# init coco dict
coco_dict = defaultdict()
# coco_dict['info'] = {}
# coco_dict['licenses'] = []
coco_dict['categories'] = []
coco_dict['images'] = []
coco_dict['annotations'] = []

# # add info
# coco_dict['info']['version'] = '1.0'
# coco_dict['info']['description'] = 'Balloon Dataset'
# coco_dict['info']['year'] = 2023
# coco_dict['info']['contributor'] = 'NULL'
# coco_dict['info']['date_created'] = '2023-06-11'

category_id = 1
category = {'id': category_id, 'name': 'balloon'}
# category = {'id': category_id, 'name': 'balloon', 'supercategory': 'object'}
coco_dict['categories'].append(category)


In [6]:

annotation_id = 1
for image_id, image_info in tqdm(annotations.items(), ncols=100, total=len(annotations)):
    # extract image id
    match = re.search(r'(\d+)_.*', image_id)
    if match:
        image_id = match.group(1)
    else:
        continue

    # get fname, size, hw
    filename = image_info['filename']
    size = image_info['size']
    img = cv2.imread(os.path.join(dataset_path, filename))
    height, width, _ = img.shape

    # add to img info
    image = {'id': int(image_id), 'file_name': filename, 'width': width, 'height': height}
    coco_dict['images'].append(image)

    # get polygon info
    regions = image_info['regions']
    for region_id, region_info in regions.items():
        # 获取多边形顶点坐标
        x = region_info['shape_attributes']['all_points_x']
        y = region_info['shape_attributes']['all_points_y']
        points = list(zip(x, y))

        # maximum edge length of the polygon
        xmin = min(x)
        xmax = max(x)
        ymin = min(y)
        ymax = max(y)
        width = xmax - xmin
        height = ymax - ymin
        # add to annotation info
        annotation = {'id': annotation_id , 'image_id': int(image_id), 'category_id': category_id, 'segmentation': [points], 'area': width * height, 'bbox': [xmin, ymin, width, height], 'iscrowd': 0}
        coco_dict['annotations'].append(annotation)
        annotation_id  += 1

# save as COCO formatted json file
with open(os.path.join(dataset_path, jsonfname), 'w') as f:
    json.dump(coco_dict, f)


100%|███████████████████████████████████████████████████████████████| 13/13 [00:00<00:00, 47.80it/s]
